# Problem Statement

Each day an airline has planes flying on the following route a flight from Los Angeles to Houston, then from Houston to New York, then New York to Miami, and finally from Miami back to Los Angeles. 

<center><img src="./Pictures/Fuelmap.png" align="center"/></center>

To fly those routes, the airline needs to purchase jet fuel for the planes, and that fuel can be purchased at any of the four airports, the ones in Los Angeles, Houston, New York and Miami. The jet fuel prices in each city is \$0.88 per gallon in Los Angeles, \$0.15 per gallon in Houston, \$1.05 per gallon in New York, and \$0.95 per gallon in Miami.

Jet fuel, like water, is heavy, and the heavier the plane is, the more jet fuel is required for it to take off and fly. So buying extra fuel in Houston might save money on purchasing, but it also means the airline would have to buy extra fuel to be able to carry all that fuel that it purchased. There's a capacity of the fuel tank. A plane can't carry more than 14,000 gallons of fuel. Second to provide a safety margin, planes need to land with at least 600 gallons of fuel still left in the tank. And third, in this example, there's a restriction that no more than 12,000 gallons of fuel can be purchased per airplane at each stop.

Note: A simplified formula for the amount of fuel used is that the number of gallons used per mile on each flight is approximately one plus 1/2000 of the average fuel level on the flight, where the average fuel level on the flight is estimated as the average of the starting and ending fuels.

# Optimization Model

**Data**
- Cities be $i$ for $i \in {LA, HOU, NY, MIA}$
- Fuel cost in city $i$ be $c_i$
- Tank capacity, safety margin, and purchase limit be $T$, $S$, and $M$ respectively
- Distance from $i$ to the next city be $d_i$

In [1]:
# Import libraries
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Read data
df = pd.read_excel("Data/jet_fuel.xlsx", sheet_name="jet-fuel")
df

,city,fuel_cost,predecessor,Unnamed: 3,legs:from,legs:to,distance,Unnamed: 7,tank_capacity,Unnamed: 9,safety_margin,Unnamed: 11,max_purchase
0,LA,0.88,MIA,NaN,LA,HOU,1500,NaN,14000.0,NaN,600.0,NaN,12000.0
1,HOU,0.15,LA,NaN,HOU,NY,1700,NaN,NaN,NaN,NaN,NaN,NaN
2,NY,1.05,HOU,NaN,NY,MIA,1300,NaN,NaN,NaN,NaN,NaN,NaN
3,MIA,0.95,NY,NaN,MIA,LA,2700,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Extract data
cities, fuel_cost, pred_city = gp.multidict(
    {city: [cost, pred] for city, cost, pred in zip(df["city"], df["fuel_cost"], df["predecessor"])}
)
tank_capacity = df["tank_capacity"].dropna().values[0]
safety_margin = df["safety_margin"].dropna().values[0]
max_purchase = df["max_purchase"].dropna().values[0]
routes, distances= gp.multidict(
    {(origin, destination): distance for origin, destination, distance in zip(df["legs:from"], df["legs:to"], df["distance"])}
)

In [4]:
### ________ Model ________
model = gp.Model("jet-fuel")

Set parameter Username


**Variables**
- Fuel purchased at city $i$ be $x_i$
- Fuel at take-off at city $i$ be $y_i$
- Fuel at landing at city $i$ be $z_i$

In [5]:
### ________ Variables ________
x = model.addVars(cities, ub=max_purchase, vtype=GRB.CONTINUOUS, name="x")
y = model.addVars(cities, ub=tank_capacity, vtype=GRB.CONTINUOUS, name="y")
z = model.addVars(cities, lb=safety_margin, vtype=GRB.CONTINUOUS, name="z")

**Objective**

Minimize total cost
$$\min \sum_ic_ix_i$$

In [6]:
### ________ Objective ________
model.setObjective(x.prod(fuel_cost), sense=GRB.MINIMIZE)

**Constraints**
- Tank capacity $y_i \le T$ for all cities $i$
- Meet minimum safety margin $z_i \ge S$ for all cities $i$
- Purchase limit at each city $x_i \le M$ for all cities $i$
- Non-negativity for all variables
- Fuel balance on ground: (Fuel landed with) + (Fuel puchased) = (Fuel taken off with) for all cities $i$
$$z_i+x_i=y_i$$
- Fuel balance on air: (Fuel taken off with at the city before $i$) - (Fuel used in flight from the city before $i$ to $i$) = (Fuel landed with at city $i$) for all cities $i$
$$y_{Pred(i)} - \left[ 1 + \frac {1} {2000} \left( \frac {y_{Pred(i)} + z_{i}} {2}  \right) \right] d_{Pred(i)} = z_i$$


In [7]:
### ________ Constraints ________
# Bounded constraints are included in variables definition
# Non-negativity is built into variables definitions as default unless specified otherwise
model.addConstrs((z[i] + x[i] == y[i] for i in cities), name="ground_balance")
model.addConstrs((y[pred_city[i]] - (1 + 0.5 * (y[pred_city[i]] + z[i]) / 2000) * distances[(pred_city[i], i)] == z[i] for i in cities), name="air_balance")

### ________ Optimize ________
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12850HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 8 rows, 12 columns and 20 nonzeros
Model fingerprint: 0x8c25b3d7
Coefficient statistics:
  Matrix range     [3e-01, 2e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [6e+02, 1e+04]
  RHS range        [1e+03, 3e+03]
Presolve removed 4 rows and 4 columns
Presolve time: 0.01s
Presolved: 4 rows, 8 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.2809734e+03   1.733792e+03   0.000000e+00      0s
       3    1.4424468e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.442446792e+04


In [8]:
print(f"Fuel cost: ${round(model.ObjVal, 0)}")
for city in cities:
    print("\n" ,"*" * 55, "\n")
    print(f"     Fuel at landing at city {city}: {round(z[city].X, 0)} gallons.")
    print(f"     Fuel purchased at city {city}: {round(x[city].X, 0)} gallons.")
    print(f"     Fuel at takeoff at city {city}: {round(y[city].X, 0)} gallons.")

Fuel cost: $14424.0

 ******************************************************* 

     Fuel at landing at city LA: 600.0 gallons.
     Fuel purchased at city LA: 3120.0 gallons.
     Fuel at takeoff at city LA: 3720.0 gallons.

 ******************************************************* 

     Fuel at landing at city HOU: 600.0 gallons.
     Fuel purchased at city HOU: 12000.0 gallons.
     Fuel at takeoff at city HOU: 12600.0 gallons.

 ******************************************************* 

     Fuel at landing at city NY: 3891.0 gallons.
     Fuel purchased at city NY: 0.0 gallons.
     Fuel at takeoff at city NY: 3891.0 gallons.

 ******************************************************* 

     Fuel at landing at city MIA: 1001.0 gallons.
     Fuel purchased at city MIA: 10399.0 gallons.
     Fuel at takeoff at city MIA: 11400.0 gallons.
